In [10]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import sagemaker
from sagemaker.experiments.run import Run, load_run
import time
from datetime import datetime
from sagemaker import image_uris

sagemaker.__version__

'2.219.0'

In [11]:
data_version = "v1"
model_version = "v1"
session = sagemaker.Session()
sm = session.sagemaker_client

experiment_name = "team1-index-predictor-model-evaluation"
trial_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
trial_name = f"model-evaluation-{model_version}-{trial_suffix}"

feature_group_name = 'index-predictor-feature-group-v7'

bucket_name = "team1-index-predictor-bucket"
training_job_name = "sagemaker-xgboost-2024-06-21-20-19-27-449"

data_input_prefix = f"data/processed/{data_version}"
model_input_prefix = f"models/{model_version}/{training_job_name}/output"
output_bucket_prefix = f"models/{model_version}/evaluation_results"

processing_instance_type = "ml.m5.large"
processing_instance_count = 1

In [12]:
with Run(
    experiment_name=experiment_name,
    run_name=trial_name,
    run_display_name=trial_name,
    sagemaker_session=session,
) as run:
    experiment_config = run.experiment_config

xgboost_container = image_uris.retrieve("xgboost", session.boto_region_name, version="1.3-1")

script_processor = ScriptProcessor(
    command=["python3"],
    image_uri=xgboost_container,
    role=sagemaker.get_execution_role(),
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
)

script_processor.run(
    code="../../src/models/evaluator.py",
    inputs=[
        ProcessingInput(
            source=f"s3://{bucket_name}/{data_input_prefix}",
            destination="/opt/ml/processing/input",
        ),
        ProcessingInput(
            source=f"s3://{bucket_name}/{model_input_prefix}",
            destination="/opt/ml/processing/model",
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=f"s3://{bucket_name}/{output_bucket_prefix}",
            output_name="evaluation_result",
        )
    ],
    arguments=[
        "--mode",
        "feature_store",
        "--input_path",
        "/opt/ml/processing/input",
        "--dataset_sizes_path",
        "/opt/ml/processing/input/dataset_sizes.json",
        "--data_version",
        data_version,
        "--target_column",
        "close_target",
        "--columns_to_drop",
        "write_time,api_invocation_time,is_deleted,datetime,type,version",
        "--model_path",
        "/opt/ml/processing/model/model",
        "--output_path",
        "/opt/ml/processing/output",
        "--feature_group_name",
        feature_group_name,
        "--region",
        "eu-central-1",
        "--bucket_name",
        bucket_name,
    ],
    experiment_config=experiment_config,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name sagemaker-xgboost-2024-06-21-20-57-27-384


.............Collecting optuna
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 

In [ ]:
!aws s3 ls s3://{bucket_name}/{output_bucket_prefix}/output/

In [13]:
!aws s3 cp s3://{bucket_name}/{output_bucket_prefix}/evaluation_report.json .

download: s3://team1-index-predictor-bucket/models/v1/evaluation_results/evaluation_report.json to ./evaluation_report.json
